In [18]:
import requests
import pandas as pd
import json
import time

## Данные по кредитам
[дом.рф/ипотечное кредитование](https://наш.дом.рф/аналитика/ипотечное_кредитование/детали/таблица)

In [19]:
def get_payload(date):
    return "{{\"regionId\":1,\"reportingPeriod\":\"{0}\",\"currency\":\"ALL\",\"creditType\":\"total\",\"periodType\":\"month\",\"regionWithAutonomous\":\"yes\"}}".format(date)

In [20]:
def get_df(date):
    url = 'https://xn--80az8a.xn--d1aqf.xn--p1ai/аналитика/apid/mortgage/api/table/data'
    headers = {'accept': 'application/json', 'content-type': 'application/json'}
    success = False
    
    while not success:
        print(f'Making request for {date}... ', end='')
        r = requests.post(url, data=get_payload(date), headers=headers)
        if r.status_code == 200:
            print('OK')
            success = True
        else:
            print(f'\nRequest failed with {r.status_code} code. Retry in 30 seconds... ')
            time.sleep(30)
            
    print('Checking data quality... ', end='')
    data = json.loads(r.text)['data']['body']
    if data:
        print('OK')
        df = pd.DataFrame(data).sort_values('sortOrder')
        df['period'] = date
        df = df.set_index(['id', 'period'])
        return df
    else:
        print(f'WARNING: data for {date} period is not available.')
        return None

In [21]:
date_range = pd.date_range(start='2019-01-01', end='2022-12-01', freq='MS')

In [22]:
df_cred = []

In [ ]:
for date in date_range:
    dt = str(date.date())
    df_cred.append(get_df(dt))

In [24]:
df_cred = pd.concat(df_cred)

In [25]:
df_cred

,,name,fo,creditCount,creditCountPct,creditCountDelta,creditCountDeltaPct,creditAmount,creditAmountPct,creditAmountDelta,creditAmountDeltaPct,wghtdAvgRate,wghtdAvgRateDelta,sortOrder
id,period,,,,,,,,,,,,,
1,2019-01-01,Российская Федерация,False,81.054,100.000000,3.007,3.85281,173.119,100.000000,24.803701,16.7237,9.88,0.03,1
17,2019-01-01,Центральный ФО,True,20.599,25.413900,1.680,8.87996,56.585,32.685600,9.611740,20.4621,9.89,0.04,2
1758,2019-01-01,Белгородская область,False,0.715,0.882128,0.082,12.95420,1.283,0.741109,0.226000,21.3813,9.82,0.10,3
1759,2019-01-01,Брянская область,False,0.611,0.753818,0.072,13.35810,1.103,0.637134,0.304000,38.0476,9.76,0.03,4
1760,2019-01-01,Владимирская область,False,0.701,0.864856,0.043,6.53495,1.234,0.712804,0.171000,16.0865,9.86,0.04,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1101,2022-12-01,Амурская область,False,1.111,0.630376,-0.094,-7.80083,5.195,0.745475,0.655000,14.4273,6.08,0.29,94
1184,2022-12-01,Магаданская область,False,0.214,0.121423,-0.130,-37.79070,0.993,0.142494,-0.314000,-24.0245,6.20,0.69,95
1106,2022-12-01,Сахалинская область,False,0.565,0.320578,-0.226,-28.57140,2.763,0.396487,-0.718000,-20.6263,6.08,-0.01,96


## Статистика по жилищному строительству
[дом.рф/жилищное строительство](https://наш.дом.рф/аналитика/показатели_жилищного_строительства)

In [26]:
def get_url(year, month,*,fo=None, rg=None):
    url = 'https://наш.дом.рф/аналитика/api/consolidated/housing_constructor/total'
    param = f'?repYear={year}&repMonth={month}' + (f'&foCd={fo}' if fo else '') + (f'&regionCode={rg}' if rg else '')
    return url + param

In [27]:
def get_json(year, month, *, fo=None, rg=None):
    success = False
    while not success:
        print(f'Making request for year {year}, month {month}, FD {fo}, region {rg}... ', end='')
        r = requests.get(get_url(year, month, fo=fo, rg=rg))
        if r.status_code == 200:
            print('OK')
            success = True
        else:
            print(f'\nRequest failed with {r.status_code} code. Retry in 30 seconds... ')
            time.sleep(30)
            
    print('Checking data quality... ', end='')
    data = json.loads(r.text)['data']
    if data:
        print('OK')
        j = data[0]
        j['period'] = f"{year}-{month:02}-01"
        j['id'] = 1000 if fo is None and rg is None else (fo if rg is None else rg)
        return j
    else:
        print(f'WARNING: data for these parameters is not available.')
        return None

In [28]:
with open('regions.json') as f:
    regions = json.load(f)    

In [29]:
date_range = pd.date_range(start='2019-01-01', end='2022-12-01', freq='MS')

In [30]:
full_json = []

In [43]:
for date in date_range:
    year = date.date().year
    month = date.date().month

#     SUBJECTS
    for fd in regions['regionsByFo']:
        for rg in fd['regions']:
            fo_code = fd['federalDistrict']['foCd']
            rg_code = rg['regionCd']
            rg_name = rg['regionShortDesc']
            
            j = get_json(year, month, rg=rg_code, fo=fo_code)
            if j is not None:
                j['name'] = rg_name
                full_json.append(j)
                
#     FEDERAL DISTRICTS
    for fd in regions['regionsByFo']:
        fo_code = fd['federalDistrict']['foCd']
        fo_name = fd['federalDistrict']['foDesc']

        j = get_json(year, month, fo=fo_code)
        if j is not None:
            j['name'] = fo_name
            full_json.append(j)

#     COUNTRY
    j = get_json(year, month)
    if j is not None:
        j['name'] = 'Российская Федерация'
        full_json.append(j)

In [35]:
df_stat = pd.DataFrame(full_json)

In [36]:
df_stat

,repYear,repMonth,type,subject,subjectDesc,devCnt,rnsCnt,rpdCnt,objCnt,livingSquareSum,...,zosgLivingElemSum,nonguarantyDevCnt,nonguarantyRnsCnt,nonguarantyRpdCnt,nonguarantyObjCnt,nonguarantyLivingSquareSum,nonguarantyLivingElemSum,period,id,name
0,2019,10,total,100,Российская Федерация,12,20,20,21,124.0,...,1.0,7,9,9,10,49.0,1.0,2019-10-01,28,Амурская область
1,2019,10,total,100,Российская Федерация,7,17,17,17,120.0,...,2.3,2,2,2,2,13.0,0.2,2019-10-01,75,Забайкальский край
2,2019,10,total,100,Российская Федерация,1,1,1,1,0.6,...,NaN,1,1,1,1,0.6,0.0,2019-10-01,41,Камчатский край
3,2019,10,total,100,Российская Федерация,44,63,63,84,853.0,...,10.3,3,4,4,7,59.0,1.2,2019-10-01,25,Приморский край
4,2019,10,total,100,Российская Федерация,11,27,29,49,285.0,...,4.5,5,5,5,14,60.0,1.4,2019-10-01,3,Республика Бурятия
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,2019,10,total,100,Российская Федерация,386,809,864,1260,9075.0,...,112.6,111,209,215,332,2024.0,38.6,2019-10-01,107,Сибирский ФО
86,2019,10,total,100,Российская Федерация,289,707,724,862,7881.0,...,82.4,62,111,112,139,1201.0,22.1,2019-10-01,106,Уральский ФО
87,2019,10,total,100,Российская Федерация,1041,2261,2320,3392,40392.0,...,531.7,221,340,338,565,3763.0,69.3,2019-10-01,101,Центральный ФО
88,2019,10,total,100,Российская Федерация,461,942,973,1819,15096.0,...,190.2,124,242,247,375,3261.0,69.8,2019-10-01,103,Южный ФО


In [37]:
df_stat = df_stat.set_index(['id', 'period'])

In [38]:
df_stat = df_stat.drop(columns=['repYear', 'repMonth', 'type', 'subject', 'subjectDesc'])

In [40]:
col_list = df_stat.columns.to_list()
reordered = col_list[-1:] + col_list[0:-1]
df_stat = df_stat[reordered]

In [41]:
with pd.ExcelWriter('data.xlsx') as f:
    df_cred.to_excel(f, sheet_name='credits')
    df_stat.to_excel(f, sheet_name='sources')